In [11]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [12]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [13]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True
    

In [14]:
# Application Layer - not core

def return_date_list(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    return_date_list = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
    return return_date_list

In [15]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2021-05-11'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-1234'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    date_list = return_date_list(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)

In [16]:
# run

main()

## Reading the uploaded file

In [17]:
trg_bucket = 'xetra-1234'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20210510_101629.parquet
xetra_daily_report_20210510_143346.parquet
xetra_daily_report_20210511_125520.parquet
xetra_daily_report_20210512_084101.parquet


In [20]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20210512_084101.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [21]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-05-11,8.85,8.82,8.82,8.85,304,-2.22
1,AT0000606306,2021-05-11,19.11,19.12,19.01,19.16,3447,-0.78
2,AT0000609607,2021-05-11,17.30,17.30,17.30,17.30,0,-0.92
3,AT0000644505,2021-05-11,111.40,114.80,111.40,115.40,313,2.68
4,AT0000652011,2021-05-11,31.39,31.73,31.35,31.73,7639,0.83
...,...,...,...,...,...,...,...,...
2938,XS2265368097,2021-05-11,15.15,15.10,15.10,15.15,33,-0.19
2939,XS2265369574,2021-05-11,22.56,22.65,22.56,22.67,26,0.09
2940,XS2265369731,2021-05-11,10.28,10.16,10.16,10.30,0,-2.73
2941,XS2265370234,2021-05-11,24.42,24.09,24.07,24.49,600,-2.03
